In [3]:
import sagemaker
from sklearn.model_selection import  train_test_split
import boto3
import pandas as pd
import numpy as np

sm_boto3 = boto3.client("sagemaker")
sess = sagemaker.Session()
region = sess.boto_session.region_name
bucket = 'mobil-e-bucket'
print("using_bucket" + bucket)

using_bucketmobil-e-bucket


In [4]:
df = pd.read_csv(r'mobile_price_prediction.csv')

In [5]:
df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1.0
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2.0
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2.0
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2.0
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1.0


In [6]:
df.shape

(3000, 21)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   battery_power  3000 non-null   int64  
 1   blue           3000 non-null   int64  
 2   clock_speed    3000 non-null   float64
 3   dual_sim       3000 non-null   int64  
 4   fc             3000 non-null   int64  
 5   four_g         3000 non-null   int64  
 6   int_memory     3000 non-null   int64  
 7   m_dep          3000 non-null   float64
 8   mobile_wt      3000 non-null   int64  
 9   n_cores        3000 non-null   int64  
 10  pc             3000 non-null   int64  
 11  px_height      3000 non-null   int64  
 12  px_width       3000 non-null   int64  
 13  ram            3000 non-null   int64  
 14  sc_h           3000 non-null   int64  
 15  sc_w           3000 non-null   int64  
 16  talk_time      3000 non-null   int64  
 17  three_g        3000 non-null   int64  
 18  touch_sc

In [8]:
# Check for no. of null values in each column
df.isnull().sum()

battery_power       0
blue                0
clock_speed         0
dual_sim            0
fc                  0
four_g              0
int_memory          0
m_dep               0
mobile_wt           0
n_cores             0
pc                  0
px_height           0
px_width            0
ram                 0
sc_h                0
sc_w                0
talk_time           0
three_g             0
touch_screen        0
wifi                0
price_range      1000
dtype: int64

In [9]:
df['price_range'] = df['price_range'].fillna(4)
df['price_range'] = df['price_range'].astype(int)

In [10]:
x = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [11]:
x.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
0,842,0,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,6,905,1988,2631,17,3,7,1,1,0
2,563,1,0.5,1,2,1,41,0.9,145,5,6,1263,1716,2603,11,2,9,1,1,0
3,615,1,2.5,0,0,0,10,0.8,131,6,9,1216,1786,2769,16,8,11,1,0,0
4,1821,1,1.2,0,13,1,44,0.6,141,2,14,1208,1212,1411,8,2,15,1,1,0


In [12]:
y.head()

0    1
1    2
2    2
3    2
4    1
Name: price_range, dtype: int32

In [13]:
y.shape

(3000,)

In [14]:
y.value_counts()

price_range
4    1000
1     500
2     500
3     500
0     500
Name: count, dtype: int64

In [15]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.15, random_state=0)

In [16]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(2550, 20)
(450, 20)
(2550,)
(450,)


In [17]:
trainX = pd.DataFrame(X_train)
trainX['price_range'] = y_train

testX = pd.DataFrame(X_test)
testX['price_range'] = y_test

In [18]:
print(trainX.shape)
print(testX.shape)

(2550, 21)
(450, 21)


In [19]:
trainX.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
422,1503,1,0.7,0,10,1,43,0.2,166,4,...,841,1304,2122,15,1,2,1,0,0,2
838,854,0,0.5,0,2,0,30,0.5,125,8,...,518,1152,2889,5,2,11,0,0,1,2
1735,753,1,0.5,0,11,1,48,0.1,121,8,...,606,1316,3484,11,10,16,1,0,0,3
746,1884,0,2.1,1,7,1,43,0.8,185,1,...,451,819,3619,14,8,7,1,1,1,3
900,1005,0,0.5,0,0,0,46,0.4,133,8,...,176,1556,3884,17,0,20,0,1,1,3


In [20]:
trainX.isnull().sum()

battery_power    0
blue             0
clock_speed      0
dual_sim         0
fc               0
four_g           0
int_memory       0
m_dep            0
mobile_wt        0
n_cores          0
pc               0
px_height        0
px_width         0
ram              0
sc_h             0
sc_w             0
talk_time        0
three_g          0
touch_screen     0
wifi             0
price_range      0
dtype: int64

In [21]:
# from sklearn.impute import KNNImputer
# impute = KNNImputer(missing_values='nan', n_neighbors=5)
# impute.fit_transform([trainX['price_range']])

In [22]:
trainX.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
422,1503,1,0.7,0,10,1,43,0.2,166,4,...,841,1304,2122,15,1,2,1,0,0,2
838,854,0,0.5,0,2,0,30,0.5,125,8,...,518,1152,2889,5,2,11,0,0,1,2
1735,753,1,0.5,0,11,1,48,0.1,121,8,...,606,1316,3484,11,10,16,1,0,0,3
746,1884,0,2.1,1,7,1,43,0.8,185,1,...,451,819,3619,14,8,7,1,1,1,3
900,1005,0,0.5,0,0,0,46,0.4,133,8,...,176,1556,3884,17,0,20,0,1,1,3


In [23]:

trainX['price_range'].value_counts()


price_range
4    863
2    437
3    419
1    419
0    412
Name: count, dtype: int64

In [24]:
trainX.to_csv("train_1.csv",index=False)
testX.to_csv("test_1.csv",index=False)

In [25]:
sk_prefix = "sagemaker/mobile_price_pred/sklearncontainer"
trainpath = sess.upload_data(
    path="train_1.csv",bucket=bucket, key_prefix=sk_prefix
)

testpath = sess.upload_data(
    path="test_1.csv",bucket=bucket, key_prefix=sk_prefix
)

In [26]:
trainX['price_range'].unique()

array([2, 3, 1, 0, 4])

In [27]:
%%writefile script.py

import xgboost
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score,classification_report, confusion_matrix, precision_score,recall_score
import sklearn
import joblib 
import boto3
import pathlib
import argparse
from io import StringIO
import os
import numpy as np
import pandas as pd

def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf

if __name__ =='__main__':

    print("[INFO] Extracting arguments")
    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the model
    parser.add_argument("--n_estimators", type=int, default=2)
    parser.add_argument("--max_depth", type=int, default=2)

    # Data,model and output directories
    parser.add_argument("--model_dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--train-file", type=str, default="train_1.csv")
    parser.add_argument("--test-file", type=str, default="test_1.csv")

    args, _ = parser.parse_known_args()

    print("SKLearn Version:", sklearn.__version__)  
    print("Joblib Version:", joblib.__version__)    

    print("[INFO] Reading data")
    print()
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))

    features = list(train_df.columns)
    label = features.pop(-1)

    print("Building training and testing datasets")
    print()
    X_train = train_df[features]
    X_test = test_df[features]
    y_train = train_df[label]
    y_test = test_df[label]

    print('Column order: ')
    print(features)
    print()

    print('Label column is:', label)
    print()

    print("Data Shape: ")
    print()
    print("---- SHAPE OF TRAINING DATA (85%) ----")
    print(X_train.shape)
    print(y_train.shape)
    print()
    print("---- SHAPE OF TESTING DATA (15%) ----")
    print(X_test.shape)
    print(y_test.shape)
    print()

    print("Training XGBoost Model.....")
    print()
    model = XGBClassifier(n_estimators=args.n_estimators, max_depth=args.max_depth)
    model.fit(X_train,y_train)
    print()

    model_path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model,model_path)
    print("Model persisted at " + model_path)
    print()

    y_pred_test = model.predict(X_test)
    test_acc = accuracy_score(y_test,y_pred_test)
    test_rep = classification_report(y_test,y_pred_test)

    print()
    print("---- METRICS RESULTS FOR TESTING DATA ----")
    print()
    print("Total Rows are: ", X_test.shape[0])
    print('[TESTING] Model Accuracy is: ', test_acc)
    print('[TESTING] Testing Report: ')
    print(test_rep)



Overwriting script.py


In [28]:
from sagemaker.xgboost.estimator import XGBoost

FRAMEWORK_VERSION = "1.0-1"
estimator = XGBoost(
    entry_point="script.py",
    role="arn:aws:iam::730335621142:role/sage-model-trainer",
    instance_count=1,
    instance_type="ml.m5.large",
    framework_version=FRAMEWORK_VERSION,
    base_job_name="XGB-custom-sklearn",
    hyperparameters={
        "n_estimators":2,
        "max_depth":2
    },
    use_spot_instances=True,  # Enable managed spot training
    max_wait=7200,  # Maximum time in seconds that training can wait for a spot instance
    max_run=3600,  # Maximum allowed runtime in seconds
)

In [29]:
# launch training job, with ascynchronous call 
estimator.fit({"train":trainpath, "test":testpath}, wait=True)

INFO:sagemaker:Creating training-job with name: XGB-custom-sklearn-2024-04-22-19-00-21-227


2024-04-22 19:00:26 Starting - Starting the training job...
2024-04-22 19:00:41 Starting - Preparing the instances for training...
2024-04-22 19:01:15 Downloading - Downloading input data...
2024-04-22 19:02:00 Downloading - Downloading the training image.....[2024-04-22 19:02:45.529 ip-10-2-82-37.ec2.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Invoking user training script.
INFO:sagemaker-containers:Module script does not provide a setup.py. 
Generating setup.py
INFO:sagemaker-containers:Generating setup.cfg
INFO:sagemaker-containers:Generating MANIFEST.in
INFO:sagemaker-containers:Installing module with the following command:
/miniconda3/bin/python3 -m pip install . 
Processing /opt/ml/code
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finish

In [30]:
# Creating artifact of model created
estimator.latest_training_job.wait(logs="None")
artifact = sm_boto3.describe_training_job(
        TrainingJobName= estimator.latest_training_job.name
)["ModelArtifacts"]["S3ModelArtifacts"]

print("Model artifacts persisted at" + artifact)


2024-04-22 19:03:03 Starting - Preparing the instances for training
2024-04-22 19:03:03 Downloading - Downloading the training image
2024-04-22 19:03:03 Training - Training image download completed. Training in progress.
2024-04-22 19:03:03 Uploading - Uploading generated training model
2024-04-22 19:03:03 Completed - Training job completed
Model artifacts persisted ats3://sagemaker-us-east-1-730335621142/XGB-custom-sklearn-2024-04-22-19-00-21-227/output/model.tar.gz


In [31]:
#Check for the artifact saved in S3 bucket
artifact

's3://sagemaker-us-east-1-730335621142/XGB-custom-sklearn-2024-04-22-19-00-21-227/output/model.tar.gz'

In [32]:
# Creating copy of model trained to make deployement
from sagemaker.sklearn.model import SKLearnModel
from time import gmtime, strftime

model_name = "Custom-sklearn-model" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model = SKLearnModel(
    name = model_name,
    model_data=artifact,
    role="arn:aws:iam::730335621142:role/sage-model-trainer",
    entry_point="script.py",
    framework_version=FRAMEWORK_VERSION
)

In [33]:
# Checking for the copy of model saved
model

In [34]:
model_name

'Custom-sklearn-model2024-04-22-19-03-32'

In [40]:
# Endpoints deployement
endpoint_name = "Custom-sklearn-model" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointsName={}".format(endpoint_name))

predictor = model.deploy(
    instance_type="ml.m4.xlarge", 
    initial_instance_count=1,
    endpoint_name=endpoint_name,
)


EndpointsName=Custom-sklearn-model2024-04-22-19-07-10


INFO:sagemaker:Creating model with name: Custom-sklearn-model2024-04-22-19-03-32
INFO:sagemaker:Creating endpoint-config with name Custom-sklearn-model2024-04-22-19-07-10
INFO:sagemaker:Creating endpoint with name Custom-sklearn-model2024-04-22-19-07-10


---